# Competitor Product Pricing Intelligence Workflow with Multi-Agent Analysis

## Overview

This notebook implements a **multi-agent workflow** for competitive intelligence analysis using Azure AI Agents with File Search.

### Workflow Architecture

```
PDF Documents → File Search Agent → Data Extractor Agent → Pricing Analyzer Agent → Report Generator Agent → Final Report
```

### Agents & Responsibilities

1. **📄 Document Search Agent**: Uses Azure File Search to query PDF catalogs
2. **📊 Data Extraction Agent**: Extracts structured product data (name, SKU, price, features)
3. **💰 Pricing Analysis Agent**: Analyzes pricing strategies, compares competitors, finds patterns
4. **📈 Insights Generator Agent**: Creates visualizations and discovers key insights
5. **📝 Report Generator Agent**: Compiles final markdown report with analysis and graphs

### Output

- **Structured Data**: JSON files with extracted products
- **Visualizations**: PNG charts (pricing distribution, feature comparison, category breakdown)
- **Final Report**: Comprehensive markdown report with analysis and embedded graphs

## Step 1: Install Dependencies

In [1]:
import sys
import subprocess

packages = [
    'python-dotenv',
    'matplotlib',
    'seaborn',
    'pandas',
    'numpy',
]

for package in packages:
    try:
        __import__(package.replace('-', '_'))
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        print(f"✅ {package} installed")

print("\n🎉 All dependencies ready!")

📦 Installing python-dotenv...
✅ python-dotenv installed
📦 Installing matplotlib...
✅ python-dotenv installed
📦 Installing matplotlib...
✅ matplotlib installed
📦 Installing seaborn...
✅ matplotlib installed
📦 Installing seaborn...
✅ seaborn installed
✅ seaborn installed
✅ pandas already installed
✅ numpy already installed

🎉 All dependencies ready!
✅ pandas already installed
✅ numpy already installed

🎉 All dependencies ready!


## Step 2: Import Libraries

In [2]:
import asyncio
import json
import os
from pathlib import Path
from typing import Optional, List, Dict, Any
from datetime import datetime
import re

# Data analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Agent framework
from agent_framework import ChatAgent, HostedFileSearchTool, HostedVectorStoreContent
from agent_framework_azure_ai import AzureAIAgentClient
from azure.ai.agents.models import FileInfo, VectorStore
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set plot style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports successful!")
print("📊 Ready for competitive intelligence workflow!")

✅ Imports successful!
📊 Ready for competitive intelligence workflow!


## Step 3: Setup Environment

In [3]:
# Create folder structure
folders = {
    'input': './competitive_analysis/input',
    'output': './competitive_analysis/output',
    'data': './competitive_analysis/data',
    'charts': './competitive_analysis/charts',
}

for folder_name, folder_path in folders.items():
    Path(folder_path).mkdir(parents=True, exist_ok=True)
    print(f"✅ {folder_name}: {folder_path}")

# Verify configuration
project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
if not project_endpoint:
    raise ValueError("❌ AZURE_AI_PROJECT_ENDPOINT not set in .env file")

print(f"\n✅ Azure AI Project: {project_endpoint}")
print("✅ Environment ready!")

✅ input: ./competitive_analysis/input
✅ output: ./competitive_analysis/output
✅ data: ./competitive_analysis/data
✅ charts: ./competitive_analysis/charts

✅ Azure AI Project: https://gk-agent-framework-project.services.ai.azure.com/api/projects/agentframworkProject
✅ Environment ready!


## Step 4: Agent 1 - Document Search & Data Extraction

This agent uses Azure File Search to extract structured product data from all PDF catalogs.

In [ ]:
async def extract_product_data():
    """
    Agent 1: Extract structured product data from all PDF catalogs using Azure File Search.
    """
    print("\n" + "="*70)
    print(" AGENT 1: DOCUMENT SEARCH & DATA EXTRACTION")
    print("="*70)
    
    project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
    client = AzureAIAgentClient(
        endpoint=project_endpoint,
        async_credential=AzureCliCredential()
    )
    
    files: List[FileInfo] = []
    vector_store: Optional[VectorStore] = None
    extracted_products = []
    
    try:
        # 1. Find PDF files
        input_path = Path(folders['input'])
        pdf_files = list(input_path.glob('*.pdf'))
        
        if not pdf_files:
            print("\n  No PDF files found!")
            print(f"Please add PDF files to: {folders['input']}")
            return []
        
        print(f"\n Found {len(pdf_files)} PDF file(s)")
        for pdf in pdf_files:
            print(f"   - {pdf.name} ({pdf.stat().st_size / 1024 / 1024:.2f} MB)")
        
        # 2. Upload all files
        print("\n  Uploading files to Azure AI...")
        file_ids = []
        for pdf_file in pdf_files:
            file = await client.project_client.agents.files.upload_and_poll(
                file_path=str(pdf_file),
                purpose="assistants"
            )
            files.append(file)
            file_ids.append(file.id)
            print(f"    {pdf_file.name} → {file.id}")
        
        # 3. Create vector store
        print("\n  Creating vector store...")
        vector_store = await client.project_client.agents.vector_stores.create_and_poll(
            file_ids=file_ids,
            name="competitive_intelligence_store"
        )
        print(f" Vector store: {vector_store.id}")
        
        # 4. Create file search tool
        file_search_tool = HostedFileSearchTool(
            inputs=[HostedVectorStoreContent(vector_store_id=vector_store.id)]
        )
        
        # 5. Create extraction agent
        print("\n Creating data extraction agent...")
        async with ChatAgent(
            chat_client=client,
            name="DataExtractionAgent",
            instructions="""
            You are a product catalog data extraction specialist.
            
            Your task: Extract ALL products from the uploaded PDF catalogs and return as a JSON array.
            
            For EACH product, extract:
            {
                "product_name": "Official product name",
                "sku": "Product SKU or model number",
                "price": "Numeric price (extract number only, no $)",
                "price_text": "Original price text as shown in catalog",
                "description": "Product description",
                "features": ["feature1", "feature2", ...],
                "category": "Product category",
                "dimensions": "Dimensions if available",
                "manufacturer": "Manufacturer/brand name",
                "source_file": "Which PDF this came from"
            }
            
            IMPORTANT:
            - Extract numeric price value for analysis (e.g., from "$1,234.56" extract 1234.56)
            - Include as many products as you can find (aim for at least 20-30 products per catalog)
            - Be thorough - scan through all pages
            - Return ONLY valid JSON array, no additional text
            """,
            tools=file_search_tool,
        ) as agent:
            # 6. Extract products
            print("\n Extracting product data from all catalogs...")
            print("   This may take a few minutes for large catalogs...")
            
            query = """
            Search through ALL uploaded PDF catalogs and extract detailed product information.
            Return a comprehensive JSON array with ALL products you can find.
            Include product_name, sku, price (numeric), price_text, description, features, 
            category, dimensions, manufacturer, and source_file for each product.
            """
            
            response = await agent.run(query)
            
            # 7. Parse JSON response
            try:
                response_text = response.text
                
                # Clean up response (remove markdown if present)
                if "```json" in response_text:
                    response_text = response_text.split("```json")[1].split("```")[0].strip()
                elif "```" in response_text:
                    response_text = response_text.split("```")[1].split("```")[0].strip()
                
                extracted_products = json.loads(response_text)
                
                print(f"\n Extracted {len(extracted_products)} products")
                
                # Save raw data
                output_file = Path(folders['data']) / "extracted_products.json"
                with open(output_file, 'w', encoding='utf-8') as f:
                    json.dump(extracted_products, f, indent=2, ensure_ascii=False)
                print(f" Saved to: {output_file}")
                
                # Show sample
                print(f"\n Sample products:")
                for i, product in enumerate(extracted_products[:3], 1):
                    print(f"\n{i}. {product.get('product_name', 'Unknown')}")
                    print(f"   SKU: {product.get('sku', 'N/A')}")
                    print(f"   Price: {product.get('price_text', 'N/A')}")
                    print(f"   Category: {product.get('category', 'N/A')}")
                
            except json.JSONDecodeError as e:
                print(f"\n  Could not parse JSON: {e}")
                # Save raw response for debugging
                output_file = Path(folders['data']) / "extraction_response.txt"
                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write(response.text)
                print(f" Saved raw response to: {output_file}")
        
    except Exception as e:
        print(f"\n Error: {e}")
        import traceback
        traceback.print_exc()
        
    finally:
        # Cleanup
        print("\n🧹 Cleaning up Azure resources...")
        try:
            if vector_store:
                await client.project_client.agents.vector_stores.delete(vector_store.id)
            for file in files:
                await client.project_client.agents.files.delete(file.id)
            if files:
                print(f" Cleaned up {len(files)} file(s) and vector store")
        except Exception:
            pass
        finally:
            await client.close()
    
    return extracted_products

# Run extraction
products_data = await extract_product_data()


🤖 AGENT 1: DOCUMENT SEARCH & DATA EXTRACTION

📄 Found 3 PDF file(s)
   - knoll-ReffProfilesVolTwo.pdf (15.99 MB)
   - haworth-tables-fixed-height_gsa-price-list-part-2.pdf (9.02 MB)
   - haworth-tables-fixed-height_gsa-price-list-part-1.pdf (11.54 MB)

⬆️  Uploading files to Azure AI...
   ✅ knoll-ReffProfilesVolTwo.pdf → assistant-YNhsGLUoufvJMbWdmMCsFc
   ✅ knoll-ReffProfilesVolTwo.pdf → assistant-YNhsGLUoufvJMbWdmMCsFc
   ✅ haworth-tables-fixed-height_gsa-price-list-part-2.pdf → assistant-825RHWgKGJTMQGTB232Pdn
   ✅ haworth-tables-fixed-height_gsa-price-list-part-2.pdf → assistant-825RHWgKGJTMQGTB232Pdn
   ✅ haworth-tables-fixed-height_gsa-price-list-part-1.pdf → assistant-WCQ3Vd5TNthkBSZc4KvfUQ

🗂️  Creating vector store...
   ✅ haworth-tables-fixed-height_gsa-price-list-part-1.pdf → assistant-WCQ3Vd5TNthkBSZc4KvfUQ

🗂️  Creating vector store...
✅ Vector store: vs_NVtwC1An9d7Jw8ffJSv61v5H

🤖 Creating data extraction agent...

🔍 Extracting product data from all catalogs...
   This 

## Step 5: Agent 2 - Pricing Analysis

This agent analyzes pricing strategies, identifies patterns, and generates insights.

In [ ]:
async def analyze_pricing(products: List[Dict[str, Any]]):
    """
    Agent 2: Analyze pricing strategies and generate insights.
    """
    print("\n" + "="*70)
    print(" AGENT 2: PRICING ANALYSIS")
    print("="*70)
    
    if not products:
        print("\n  No product data available")
        return None
    
    project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
    client = AzureAIAgentClient(
        endpoint=project_endpoint,
        async_credential=AzureCliCredential()
    )
    
    try:
        # Prepare data summary for analysis
        products_summary = json.dumps(products, indent=2)
        
        print(f"\n Creating pricing analysis agent...")
        async with ChatAgent(
            chat_client=client,
            name="PricingAnalysisAgent",
            instructions="""
            You are a senior pricing strategy analyst specializing in competitive intelligence.
            
            Your task: Analyze the provided product pricing data and generate comprehensive insights.
            
            Provide analysis on:
            1. PRICING DISTRIBUTION: Min, max, average, median prices overall and by category
            2. PRICING STRATEGIES: Identify pricing tiers (budget, mid-range, premium)
            3. COMPETITIVE POSITIONING: Compare manufacturers/brands on pricing
            4. PRICE-FEATURE CORRELATION: Analyze if higher prices correlate with more features
            5. OUTLIERS: Identify unusually expensive or cheap products
            6. RECOMMENDATIONS: Pricing strategy recommendations based on analysis
            
            Return analysis as structured markdown with clear sections.
            """,
        ) as agent:
            print("\n Analyzing pricing data...")
            
            query = f"""
            Analyze the following product pricing data and provide comprehensive insights:
            
            {products_summary[:10000]}  # Limit to first 10k chars to avoid token limits
            
            Provide detailed pricing analysis with statistics and strategic insights.
            """
            
            response = await agent.run(query)
            analysis = response.text
            
            # Save analysis
            output_file = Path(folders['data']) / "pricing_analysis.md"
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(analysis)
            
            print(f"\n✅ Pricing analysis complete")
            print(f"✅ Saved to: {output_file}")
            
            return analysis
            
    finally:
        await client.close()

# Run pricing analysis
pricing_analysis = await analyze_pricing(products_data)


💰 AGENT 2: PRICING ANALYSIS

🤖 Creating pricing analysis agent...

🔍 Analyzing pricing data...

✅ Pricing analysis complete
✅ Saved to: competitive_analysis/data/pricing_analysis.md

✅ Pricing analysis complete
✅ Saved to: competitive_analysis/data/pricing_analysis.md


## Step 6: Agent 3 - Generate Visualizations

Create charts and graphs for the competitive analysis report.

In [6]:
def generate_visualizations(products: List[Dict[str, Any]]):
    """
    Agent 3: Generate visualizations for competitive intelligence report.
    """
    print("\n" + "="*70)
    print("📈 AGENT 3: VISUALIZATION GENERATOR")
    print("="*70)
    
    if not products:
        print("\n⚠️  No product data available")
        return []
    
    # Convert to DataFrame
    df = pd.DataFrame(products)
    
    # Clean price data (extract numeric values)
    if 'price' in df.columns:
        df['price_numeric'] = pd.to_numeric(df['price'], errors='coerce')
    else:
        print("\n⚠️  No price data available")
        return []
    
    # Remove rows with invalid prices
    df = df[df['price_numeric'].notna()]
    
    if len(df) == 0:
        print("\n⚠️  No valid price data available")
        return []
    
    print(f"\n📊 Generating visualizations for {len(df)} products...")
    
    charts = []
    
    # Chart 1: Price Distribution Histogram
    plt.figure(figsize=(12, 6))
    plt.hist(df['price_numeric'], bins=30, edgecolor='black', alpha=0.7)
    plt.xlabel('Price ($)', fontsize=12)
    plt.ylabel('Number of Products', fontsize=12)
    plt.title('Product Price Distribution', fontsize=14, fontweight='bold')
    plt.grid(axis='y', alpha=0.3)
    chart1_path = Path(folders['charts']) / 'price_distribution.png'
    plt.savefig(chart1_path, dpi=300, bbox_inches='tight')
    plt.close()
    charts.append(('Price Distribution', str(chart1_path)))
    print(f"   ✅ {chart1_path.name}")
    
    # Chart 2: Price by Category (if category data exists)
    if 'category' in df.columns and df['category'].notna().any():
        plt.figure(figsize=(14, 6))
        category_prices = df.groupby('category')['price_numeric'].mean().sort_values(ascending=False)
        category_prices.plot(kind='bar', color='steelblue', edgecolor='black')
        plt.xlabel('Category', fontsize=12)
        plt.ylabel('Average Price ($)', fontsize=12)
        plt.title('Average Price by Product Category', fontsize=14, fontweight='bold')
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', alpha=0.3)
        chart2_path = Path(folders['charts']) / 'price_by_category.png'
        plt.savefig(chart2_path, dpi=300, bbox_inches='tight')
        plt.close()
        charts.append(('Price by Category', str(chart2_path)))
        print(f"   ✅ {chart2_path.name}")
    
    # Chart 3: Price by Manufacturer (if manufacturer data exists)
    if 'manufacturer' in df.columns and df['manufacturer'].notna().any():
        plt.figure(figsize=(12, 6))
        mfr_data = df.groupby('manufacturer').agg({
            'price_numeric': ['mean', 'count']
        }).reset_index()
        mfr_data.columns = ['manufacturer', 'avg_price', 'count']
        mfr_data = mfr_data[mfr_data['count'] >= 3].sort_values('avg_price', ascending=False)
        
        if len(mfr_data) > 0:
            plt.bar(range(len(mfr_data)), mfr_data['avg_price'], color='coral', edgecolor='black')
            plt.xticks(range(len(mfr_data)), mfr_data['manufacturer'], rotation=45, ha='right')
            plt.xlabel('Manufacturer', fontsize=12)
            plt.ylabel('Average Price ($)', fontsize=12)
            plt.title('Average Price by Manufacturer', fontsize=14, fontweight='bold')
            plt.grid(axis='y', alpha=0.3)
            chart3_path = Path(folders['charts']) / 'price_by_manufacturer.png'
            plt.savefig(chart3_path, dpi=300, bbox_inches='tight')
            plt.close()
            charts.append(('Price by Manufacturer', str(chart3_path)))
            print(f"   ✅ {chart3_path.name}")
    
    # Chart 4: Price Range Box Plot
    plt.figure(figsize=(10, 6))
    plt.boxplot(df['price_numeric'], vert=True, patch_artist=True,
                boxprops=dict(facecolor='lightblue', edgecolor='black'),
                medianprops=dict(color='red', linewidth=2))
    plt.ylabel('Price ($)', fontsize=12)
    plt.title('Price Range Distribution (Box Plot)', fontsize=14, fontweight='bold')
    plt.grid(axis='y', alpha=0.3)
    chart4_path = Path(folders['charts']) / 'price_boxplot.png'
    plt.savefig(chart4_path, dpi=300, bbox_inches='tight')
    plt.close()
    charts.append(('Price Box Plot', str(chart4_path)))
    print(f"   ✅ {chart4_path.name}")
    
    # Chart 5: Product Count by Category
    if 'category' in df.columns and df['category'].notna().any():
        plt.figure(figsize=(10, 8))
        category_counts = df['category'].value_counts()
        plt.pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%',
                startangle=90, colors=sns.color_palette('Set3'))
        plt.title('Product Distribution by Category', fontsize=14, fontweight='bold')
        chart5_path = Path(folders['charts']) / 'category_distribution.png'
        plt.savefig(chart5_path, dpi=300, bbox_inches='tight')
        plt.close()
        charts.append(('Category Distribution', str(chart5_path)))
        print(f"   ✅ {chart5_path.name}")
    
    print(f"\n✅ Generated {len(charts)} visualization(s)")
    return charts

# Generate visualizations
charts = generate_visualizations(products_data)


📈 AGENT 3: VISUALIZATION GENERATOR

📊 Generating visualizations for 19 products...
   ✅ price_distribution.png
   ✅ price_by_category.png
   ✅ price_distribution.png
   ✅ price_by_category.png
   ✅ price_by_manufacturer.png
   ✅ price_boxplot.png
   ✅ category_distribution.png

✅ Generated 5 visualization(s)
   ✅ price_by_manufacturer.png
   ✅ price_boxplot.png
   ✅ category_distribution.png

✅ Generated 5 visualization(s)


## Step 7: Agent 4 - Generate Final Report

Create a comprehensive markdown report with all analysis and visualizations.

In [7]:
async def generate_final_report(products: List[Dict[str, Any]], pricing_analysis: str, charts: List[tuple]):
    """
    Agent 4: Generate comprehensive final report with analysis and visualizations.
    """
    print("\n" + "="*70)
    print("📝 AGENT 4: FINAL REPORT GENERATOR")
    print("="*70)
    
    if not products:
        print("\n⚠️  No product data available")
        return
    
    # Calculate statistics
    df = pd.DataFrame(products)
    if 'price' in df.columns:
        df['price_numeric'] = pd.to_numeric(df['price'], errors='coerce')
        df = df[df['price_numeric'].notna()]
    
    print("\n📊 Generating final report...")
    
    # Generate report
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    report = f"""# Competitive Intelligence Report

**Generated:** {timestamp}  
**Products Analyzed:** {len(products)}  
**Catalogs Processed:** {len(df['source_file'].unique()) if 'source_file' in df.columns else 'N/A'}

---

## Executive Summary

This report provides a comprehensive competitive intelligence analysis of furniture products from multiple manufacturer catalogs. The analysis includes pricing strategies, feature comparisons, market positioning, and actionable recommendations.

### Key Findings

- **Total Products Analyzed:** {len(products)}
- **Price Range:** ${df['price_numeric'].min():.2f} - ${df['price_numeric'].max():.2f}
- **Average Price:** ${df['price_numeric'].mean():.2f}
- **Median Price:** ${df['price_numeric'].median():.2f}
- **Categories:** {', '.join(df['category'].unique()[:5]) if 'category' in df.columns else 'N/A'}

---

## 1. Pricing Analysis

{pricing_analysis if pricing_analysis else 'Pricing analysis not available.'}

---

## 2. Visual Analysis

"""
    
    # Add charts to report
    for chart_name, chart_path in charts:
        rel_path = Path(chart_path).relative_to(Path(folders['output']).parent)
        report += f"""### {chart_name}

![{chart_name}]({rel_path})

"""
    
    # Add data insights
    report += f"""
---

## 3. Product Insights

### Top 10 Most Expensive Products

| Rank | Product | SKU | Price | Category |
|------|---------|-----|-------|----------|
"""
    
    top_products = df.nlargest(10, 'price_numeric')
    for i, (_, product) in enumerate(top_products.iterrows(), 1):
        report += f"| {i} | {product.get('product_name', 'N/A')} | {product.get('sku', 'N/A')} | ${product['price_numeric']:.2f} | {product.get('category', 'N/A')} |\n"
    
    report += f"""
### Top 10 Most Affordable Products

| Rank | Product | SKU | Price | Category |
|------|---------|-----|-------|----------|
"""
    
    bottom_products = df.nsmallest(10, 'price_numeric')
    for i, (_, product) in enumerate(bottom_products.iterrows(), 1):
        report += f"| {i} | {product.get('product_name', 'N/A')} | {product.get('sku', 'N/A')} | ${product['price_numeric']:.2f} | {product.get('category', 'N/A')} |\n"
    
    # Add recommendations section
    project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
    client = AzureAIAgentClient(
        endpoint=project_endpoint,
        async_credential=AzureCliCredential()
    )
    
    try:
        print("\n🤖 Generating strategic recommendations...")
        async with ChatAgent(
            chat_client=client,
            name="ReportGeneratorAgent",
            instructions="""
            You are a strategic business analyst.
            Based on competitive intelligence data, generate actionable recommendations.
            Format response as clear, numbered recommendations with specific insights.
            """,
        ) as agent:
            query = f"""
            Based on this competitive intelligence summary, provide 5-7 strategic recommendations:
            
            - Total Products: {len(products)}
            - Price Range: ${df['price_numeric'].min():.2f} - ${df['price_numeric'].max():.2f}
            - Average Price: ${df['price_numeric'].mean():.2f}
            - Categories: {', '.join(df['category'].unique()[:10]) if 'category' in df.columns else 'Various'}
            
            Provide specific, actionable recommendations for:
            1. Pricing strategy
            2. Product positioning
            3. Market opportunities
            4. Competitive advantages
            """
            
            response = await agent.run(query)
            recommendations = response.text
            
            report += f"""
---

## 4. Strategic Recommendations

{recommendations}

---

## 5. Methodology

This competitive intelligence report was generated using a multi-agent AI workflow:

1. **Document Search Agent:** Extracted product data from PDF catalogs using Azure AI File Search
2. **Data Extraction Agent:** Structured and normalized product information
3. **Pricing Analysis Agent:** Analyzed pricing strategies and market positioning
4. **Visualization Agent:** Generated statistical charts and graphs
5. **Report Generator Agent:** Compiled comprehensive analysis with recommendations

### Data Sources

"""
            
            if 'source_file' in df.columns:
                for source in df['source_file'].unique():
                    count = len(df[df['source_file'] == source])
                    report += f"- {source}: {count} products\n"
            
            report += f"""
---

## Appendix: Full Product List

Complete product data available in: `{folders['data']}/extracted_products.json`

---

*Report generated by Azure AI Agents - Competitive Intelligence Workflow*
"""
    
    finally:
        await client.close()
    
    # Save report
    report_path = Path(folders['output']) / f"competitive_intelligence_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(report)
    
    print(f"\n✅ Final report generated")
    print(f"✅ Saved to: {report_path}")
    print(f"\n📊 Report includes:")
    print(f"   - Executive summary with key metrics")
    print(f"   - Detailed pricing analysis")
    print(f"   - {len(charts)} visualization(s)")
    print(f"   - Product insights (top/bottom products)")
    print(f"   - Strategic recommendations")
    
    return report_path

# Generate final report
report_path = await generate_final_report(products_data, pricing_analysis, charts)


📝 AGENT 4: FINAL REPORT GENERATOR

📊 Generating final report...

🤖 Generating strategic recommendations...

✅ Final report generated
✅ Saved to: competitive_analysis/output/competitive_intelligence_report_20251003_184657.md

📊 Report includes:
   - Executive summary with key metrics
   - Detailed pricing analysis
   - 5 visualization(s)
   - Product insights (top/bottom products)
   - Strategic recommendations

✅ Final report generated
✅ Saved to: competitive_analysis/output/competitive_intelligence_report_20251003_184657.md

📊 Report includes:
   - Executive summary with key metrics
   - Detailed pricing analysis
   - 5 visualization(s)
   - Product insights (top/bottom products)
   - Strategic recommendations


## Summary

### 🎉 Workflow Complete!

The multi-agent competitive intelligence workflow has successfully:

✅ **Extracted product data** from PDF catalogs using Azure AI File Search  
✅ **Analyzed pricing strategies** with AI-powered insights  
✅ **Generated visualizations** (distribution charts, category analysis, comparisons)  
✅ **Created comprehensive report** with analysis and recommendations  

### 📂 Output Files

- **Data**: `./competitive_analysis/data/extracted_products.json`
- **Analysis**: `./competitive_analysis/data/pricing_analysis.md`
- **Charts**: `./competitive_analysis/charts/*.png`
- **Final Report**: `./competitive_analysis/output/competitive_intelligence_report_*.md`

### 🚀 Next Steps

1. Review the final report in the output folder
2. Examine the visualizations in the charts folder
3. Use the extracted JSON data for further analysis
4. Add more PDF catalogs and re-run the workflow

### 🔧 Customization

You can customize:
- Agent instructions for different analysis focus
- Visualization types and styles
- Report sections and formatting
- Add new agents for additional analysis (feature comparison, market trends, etc.)